In [119]:
import pandas as pd
import numpy as np
import json
from random import randint
from collections import defaultdict

In [120]:
fileName = "2018-07-International-Exchange-Index"

In [127]:
csvData = pd.read_csv(fileName+".csv", dtype='str')

,ASN-source,ASN,Name,Type,Bandwidth,Gb/s,Connectivity Type
0,AS38566,AS2914,NTT (SG),ISP-Inter,350,Mbps.,Transit/IPv6 Dual Stack
1,AS4651,AS45758,Triple T Internet/ Triple T Broadband,ISP-Local,500,Mbps,IPv4
2,AS4651,AS38450,GIN,ISP-Govt,1,Gbps,IPv4
3,AS4651,AS18013,B Network Solution (BNS),ISP-Local,10,Mbps,IPv4
4,AS4651,AS9931,CAT-ISP,ISP-Local,200,Gbps,IPv4
5,AS4651,AS131090,CAT ON Net,ISP-Local,210,Gbps,IPv4
6,AS4651,AS4618,INET,ISP-Local,1.8,Gbps,IPv4
7,AS4651,AS24187,KIRZ,ISP-Local,720,Mbps,IPv4
8,AS4651,AS56067,Metrabyte,ISP-Local,101,Mbps,IPv4
9,AS4651,AS38888,MILCOM,ISP-Local,600,Mbps,IPv4


In [122]:
namedAsn = set(csvData['ASN'].values)

asnSource = set(csvData['ASN-source'].values)
unnamedAsnSource = asnSource - namedAsn
namedAsnSource = asnSource - unnamedAsnSource

mapping = {}
for asn in namedAsnSource:
    mapping[asn] = csvData.loc[csvData['ASN']==asn]['Name'].values[0]
for asn in unnamedAsnSource:
    mapping[asn] = asn

In [123]:
def new_node(color, label, attributes, x, y, _id):
    node = {}
    node["color"] = color
    node["label"] = label
    node["attributes"] = attributes
    node["y"] = y
    node["x"] = x
    node["id"] = _id
    node["size"] = 5
    return node

def new_edge(sourceID, attributes, targetID, size):
    edge = {}
    edge["sourceID"] = sourceID
    edge["attributes"] = attributes
    edge["targetID"] = targetID
    edge["size"] = size
    return edge

colors = [
"#C91F37",
"#DC3023",
"#9D2933",
"#CF000F",
"#E68364",
"#F22613",
"#CF3A24",
"#C3272B",
"#8F1D21",
"#D24D57",
"#F08F907",
"#F47983",
"#DB5A6B",
"#C93756",
"#FCC9B9",
"#FFB3A7",
"#F62459",
"#F58F84",
"#875F9A",
"#5D3F6A",
"#89729E",
"#763568",
"#8D608C",
"#A87CA0",
"#5B3256",
"#BF55EC",
"#8E44AD",
"#9B59B6",
"#BE90D4",
"#4D8FAC",
"#5D8CAE",
"#22A7F0",
"#19B5FE",
"#59ABE3",
"#48929B",
"#317589",
"#89C4F4",
"#4B77BE",
"#1F4788",
"#003171",
"#044F67",
"#264348",
"#7A942E",
"#8DB255",
"#5B8930",
"#6B9362",
"#407A52",
"#006442",
"#87D37C",
"#26A65B",
"#26C281",
"#049372",
"#2ABB9B",
"#16A085",
"#36D7B7",
"#03A678",
"#4DAF7C",
"#D9B611",
"#F3C13A",
"#F7CA18",
"#E2B13C",
"#A17917",
"#F5D76E",
"#F4D03F",
"#FFA400",
"#E08A1E",
"#FFB61E",
"#FAA945",
"#FFA631",
"#FFB94E",
"#E29C45",
"#F9690E",
"#CA6924",
"#F5AB35",
"#BFBFBF",
"#F2F1EF",
"#BDC3C7",
"#ECF0F1",
"#D2D7D3",
"#757D75",
"#EEEEEE",
"#ABB7B7",
"#6C7A89",
"#95A5A6"]
colors_count = len(colors)

In [124]:
nodes = []
edges = []
isNodeExist = defaultdict(bool)
isEdgeExist = defaultdict(bool)

for index, row in csvData.iterrows():
    label = ''
    if not isNodeExist[row['ASN-source']]:
        label = mapping[row['ASN-source']]
            
        nodes.append(
            new_node(
                    color=colors[randint(0, colors_count-1)], 
                    label=label, 
                    attributes={}, 
                    x=randint(-5, 5), 
                    y=randint(-5, 5), 
                    _id=label))
        isNodeExist[row['ASN-source']] = True
        
    if not isNodeExist[row['ASN']]:
        nodes.append(
            new_node(
                color=colors[randint(0, colors_count-1)], 
                label=row['Name'], 
                attributes={}, 
                x=randint(-5, 5), 
                y=randint(-5, 5), 
                _id=row['Name']))
        isNodeExist[row['ASN']] = True
    
    edges.append(
        new_edge(
            sourceID=mapping[row['ASN-source']], 
            attributes={}, 
            targetID=row['Name'], 
            size=1))

len(edges), len(nodes)

(637, 260)

In [125]:
graph = {}
graph["nodes"] = nodes
graph["edges"] = edges

In [126]:
with open(fileName+".json", 'w') as outfile:
    json.dump(graph, outfile, indent=4, separators=(',', ': '), sort_keys=True)